In [4]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python",trust_remote_code=True)

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [5]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [6]:
print(raw_datasets["train"][123456]["whole_func_string"])

def last_rate_limit(self):
        """
        A `dict` of the rate limit information returned in the most recent
        response, or `None` if no requests have been made yet.  The `dict`
        consists of all headers whose names begin with ``"RateLimit"`` (case
        insensitive).

        The DigitalOcean API specifies the following rate limit headers:

        :var string RateLimit-Limit: the number of requests that can be made
            per hour
        :var string RateLimit-Remaining: the number of requests remaining until
            the limit is reached
        :var string RateLimit-Reset: the Unix timestamp for the time when the
            oldest request will expire from rate limit consideration
        """
        if self.last_response is None:
            return None
        else:
            return {k:v for k,v in iteritems(self.last_response.headers)
                        if k.lower().startswith('ratelimit')}


In [8]:
training_corpus =  (
    raw-datasets["train"][i:i+1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)
)

In [13]:
def get_training_corpus():
    return (
        raw_datasets["train"][i:i+1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)
    )
training_corpus = get_training_corpus()

In [14]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [15]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''
tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [16]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 5200)

In [17]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'num',
 'bers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [18]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠL',
 'ine',
 'ar',
 'Lay',
 'er',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġt',
 'or',
 'ch',
 '.',
 'rand',
 'n',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'b',
 'ias',
 'Ġ=',
 'Ġt',
 'or',
 'ch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'b',
 'ias',
 'ĊĠĠĠĠ']

In [19]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer\\tokenizer_config.json',
 'code-search-net-tokenizer\\special_tokens_map.json',
 'code-search-net-tokenizer\\vocab.json',
 'code-search-net-tokenizer\\merges.txt',
 'code-search-net-tokenizer\\added_tokens.json',
 'code-search-net-tokenizer\\tokenizer.json')

In [20]:
from huggingface_hub import notebook_login
notebook_login()